In [9]:
import json
import cv2
import numpy as np
import os

# Mapa de colores en BGR para las clases
label_colors = {
    "sheep": (255, 0, 0),   # Rojo
    "wolf": (0, 255, 0),  # Verde
    "person": (0, 0, 255),  # Azul
    "dog": (255, 255, 0),# Cian
}


In [11]:
def build_video(dir, video_path):
  # Rutas de los directorios
  image_dir = dir
  json_dir = dir
  output_dir = os.path.join(dir, 'video_labels')

  # Asegúrate de que el directorio de salida existe
  if not os.path.exists(output_dir):
      os.makedirs(output_dir)

  # Listar todos los archivos JSON
  json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

  for json_file in json_files:
      with open(os.path.join(json_dir, json_file), 'r') as f:
          data = json.load(f)
      
      # Cargar la imagen correspondiente
      image_path = os.path.join(image_dir, data['imagePath'])
      image = cv2.imread(image_path)
      
      # Procesar cada forma (anotación)
      for shape in data['shapes']:
          label = shape['label']
          points = np.array(shape['points'], np.int32)
          color = label_colors.get(label, (255, 255, 255))  # Color blanco por defecto
          
          # Dibujar el polígono
          cv2.polylines(image, [points], isClosed=True, color=color, thickness=2)
      
      # Guardar la imagen modificada
      output_image_path = os.path.join(output_dir, os.path.basename(image_path))
      cv2.imwrite(output_image_path, image)
  
  ## Generate video
  video_codec = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para el vídeo
  video_fps = 30  # FPS del vídeo
  images = [os.path.join(output_dir, img) for img in sorted(os.listdir(output_dir)) if img.endswith(".jpg")]

  # Asumiendo que todas las imágenes tienen el mismo tamaño
  frame = cv2.imread(images[0])
  height, width, layers = frame.shape
  video = cv2.VideoWriter(video_path, video_codec, video_fps, (width, height))

  for image in images:
      video.write(cv2.imread(image))

  video.release()


In [39]:
list_dir = ['Manolo/1,2,4_correct', 'Manolo/3_correct', 'Otras/otras_correct', 'Septiembre/1-3_correct', 'Septiembre/4_correct', 'Septiembre/5_correct']
list_video_path = ['videos/Manolo_1,2,4.mp4', 'videos/Manolo_3.mp4', 'videos/otras.mp4', 'videos/Septiembre_1-3.mp4', 'videos/Septiembre_4.mp4', 'videos/Septiembre_5.mp4', 'videos/Septiembre']
for dir, video_path in zip(list_dir, list_video_path):
  build_video(dir, video_path)

In [12]:
build_video('Agosto/1-3_6-10_correct', 'videos/Agosto_1-3_6-10.mp4')